In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import KFold, StratifiedKFold
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, AUC, BinaryAccuracy
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
import random
from skimage import color
from tensorflow.keras.layers import Concatenate, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dense
from PIL import Image, ImageDraw 
import tarfile
import nibabel as nib
import matplotlib.gridspec as gridspec

In [ ]:
ROOT_PATH = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/"
TRAIN = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
TRAIN = TRAIN.loc[~TRAIN.BraTS21ID.isin([109, 123, 709])]
TRAIN.reset_index(drop=True, inplace=True)

In [ ]:
WIDTH_RESIZE = 200
COUNT_IMAGE = 4
WIDTH_IMAGE = 400
BATCH = 2
N_FOLDS = 5
EPOCHS=10

In [ ]:
def getpath(ids):
    ids = str(ids)
    len1 = len(ids)
    return "".join([*["0" for _ in range(5-len1)],ids])
    

In [ ]:
def get_maximum_image(ids,types="T1wCE"):
    path=ROOT_PATH+"train"+"/"+getpath(ids)+"/"+types+"/"
    dicom = sorted(os.listdir(path), key=lambda v:int(v.split("-")[1][:-4]))
    image = []
    for p in dicom:
#         imgd = findArea(path+p)
        di = pydicom.read_file(path+p)
        imgd = di.pixel_array
        if di.PhotometricInterpretation == "MONOCHROME1":
            imgd = np.amax(imgd) - imgd
        imgd = imgd - np.min(imgd)
        imgd = imgd / np.max(imgd)
        imgd = (imgd * 255).astype(np.uint8)
        if not type(imgd)== bool:
            
            image.append(imgd)
    count_image = len(image)//COUNT_IMAGE
    return_image = []
    for i in range(COUNT_IMAGE):
        images = image[i*count_image:(i+1)*count_image]
        if types=="T1w":   
            im = tf.keras.layers.Maximum()(images).numpy()
        else:
            im = tf.keras.layers.Maximum()(images).numpy()
            im = cv2.resize(im, (WIDTH_RESIZE,WIDTH_RESIZE))
        return_image.append(im)
    return return_image

In [ ]:
TRAIN.head(25)

In [ ]:
def construct_image(im1):
#     image_ret = np.zeros((WIDTH_IMAGE,WIDTH_IMAGE))
    image_ret2 = np.zeros((WIDTH_IMAGE,WIDTH_IMAGE,3))
    for i, arr_img in enumerate(im1):
        x,y = 0,0
        for img in arr_img:
            image_ret2[x:WIDTH_RESIZE+x,y:WIDTH_RESIZE+y,i] = img
            x+=WIDTH_RESIZE
            if x>WIDTH_IMAGE-WIDTH_RESIZE:
                x=0
                y+=WIDTH_RESIZE
    #     image_ret2[:,:,0] = image_ret
    #     image_ret2[:,:,1] = image_ret
    #     image_ret2[:,:,2] = image_ret
    return image_ret2

In [ ]:
image1 = get_maximum_image(0,types="T1wCE")
image2 = get_maximum_image(0,types="T2w")
image3 = get_maximum_image(0,types="FLAIR")
all_image = [image1, image2, image3]

In [ ]:
img = construct_image(all_image)
plt.imshow(img)

In [ ]:
def custom_generator(dataframe):
    shape = dataframe.shape[0]
    
    count = shape//BATCH
    types2 = ["FLAIR","T1w","T1wCE","T2w"]
    i=0
    while True:
            data = np.array(dataframe.loc[i*BATCH:((i+1)*BATCH)-1].values)
            label = data[:,1:2].astype(float)
#             display(dataframe.loc[i*BATCH:((i+1)*BATCH)-1])
            users =  data[:,0:1]
            imagess = data[:,2:3]
            data_1 = []
            for ii, u in enumerate(users):
                image1 = get_maximum_image(u[0],types="T1wCE")
                image2 = get_maximum_image(u[0],types="T2w")
                image3 = get_maximum_image(u[0],types="FLAIR")
                all_image = [image1, image2, image3]
                data_1.append(construct_image(all_image))
            data_1 = np.array(data_1).astype(float)
            i+=1
            if shape%2>0 and i>count:
                i=0
            if shape%2==0 and i==count:
                i=0
#             return data_1, label
            yield data_1, label

In [ ]:
!pip install efficientnet
import efficientnet.tfkeras as efn

In [ ]:
def model():
    METRICS = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'), 
      BinaryAccuracy(name='accuracy'),
      AUC(name='auc')
    ]  
    img1 = tf.keras.layers.Input(shape=(WIDTH_IMAGE,WIDTH_IMAGE,3))
    
    constructor2 = getattr(efn, f'EfficientNetB{1}')
    x2 = constructor2(include_top=False,
                        weights= "imagenet",
                        input_shape=(WIDTH_IMAGE,WIDTH_IMAGE,3),
                        pooling = "avg"
                       )(img1)
    out = Dense(1, activation="sigmoid")(x2)
    model = Model(inputs = [img1], outputs=[out])
    model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
                  metrics=METRICS
                 )
    return model

In [ ]:
model().summary()

In [ ]:
seed=42
skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=seed)
valid_preds = [];
valid_labels = [];
history_list = [];
test_preds = []
for fold,(train_idx,valid_idx) in enumerate(skf.split(TRAIN)):
    print(f'\nFOLD: {fold+1}')
    print(f'TRAIN: {len(train_idx)} VALID: {len(valid_idx)}')
#     K.clear_session()
    model = model()
    train_inputs=TRAIN.loc[train_idx].copy()
    valid_inputs = TRAIN.loc[valid_idx].copy()
    train_inputs.reset_index(inplace=True, drop=True)
    valid_inputs.reset_index(inplace=True, drop=True)
    train_g = custom_generator(train_inputs)
    val_g =  custom_generator(valid_inputs)
 
    history = model.fit_generator(train_g,
                              steps_per_epoch=train_inputs.shape[0]//BATCH,
                              epochs=EPOCHS,
#                               callbacks = [lzr_schedule],
                              validation_data = val_g,
                              validation_steps=(valid_inputs.shape[0]//BATCH)
                             )